## Exporting all the required library

In [1]:
from bs4 import BeautifulSoup
from urllib import request as urlrequest
from urllib.request import urlopen
import os
import sys
import requests
from lxml.html import fromstring
import urllib
from random import random


In [2]:
## get proxies from a website that are free and try to use these during web scrapping
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:500]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies

## gets all the hyperlinks that are in a webpage
def get_href(full_url, proxy_list):
    soup = url_to_beautifulsoup(full_url, proxy_list, timeout = 10)
    all_href = soup.find_all("a")
    links = []
    for link in all_href:
        links.append(link.get('href'))
    return links
    
##converts the given url to a beautiful soup object and uses proxy to query the url
def url_to_beautifulsoup(full_url, proxy_list, timeout = 10):
   
    proxy = proxy_list[0]
    count = 0
    while(count < 50):
        try:
            content = requests.get(full_url,proxies={"http": proxy, "https": proxy}, timeout=timeout)
            soup = BeautifulSoup(content.text)
            if "Forbidden" in str(soup.find_all("title")[0]):
                count = count + 1
                proxy = proxy_list[int(random() * len(proxy_list))]
            else:
                return soup
        except requests.exceptions.RequestException as err:
            print("Connection Error or Timeout:" + str(err))
            count = count + 1
            proxy = proxy_list[int(random() * len(proxy_list))]
        except:
            print("Abnormal error when extracting: %s" %(full_url))
            count = count + 1
            proxy = proxy_list[int(random() * len(proxy_list))]
                            
    print("Reached 50 trials for connections.")
    return None
    


 

In [3]:
## Code to gather information about the paper 
    
output_abstract_path = "full_paper_lenght.dat"

dict_keys= ['Id',
            'og:title',
            'og:site_name',
            'citation_online_date',
            'citation_title',
            'citation_author',
            'citation_pdf_url',
            'citation_arxiv_id',
            'og:description',
            'twitter:site',
            'citation_date',
            'twitter:title',
            'twitter:description',
            'og:url',
            'category']

delimiter = "||||"



def find_category(url):
    for link in get_href(url):
        if "?context" in link:
            return(link.split("=")[1])
            break
            
            
            
def extract_paper_information(full_link, proxy):
    info_dict = {}
    
    separator_within_key = " || "
    
    
    soup = url_to_beautifulsoup(full_link, proxy)
    if soup == None:
        return None
        
    all_href = soup.find_all("a")
    links = []
    category = "None"
    for link in all_href:
        if "?context" in link.get('href'):
            category = link.get('href').split("=")[1]
            break    
    info_dict ["category"] = category
    meta = soup.find_all("meta")
    for meta_scan in meta:
        content_meta = meta_scan.get("content")
        property_meta = meta_scan.get("property")
        name_meta = meta_scan.get("name")
        if (property_meta != None):
            key = property_meta
        elif (name_meta != None):
            key = name_meta
        else:
            continue
            
        key_in_info_dict = info_dict.keys()
        if key in key_in_info_dict:
            initial_value = info_dict[key]
            current_value = content_meta
            joined_value = separator_within_key.join([initial_value,current_value])
            info_dict[key] = joined_value
        else:
            info_dict[key] = content_meta
            
        
    return info_dict


def write_to_file(file,info_dict,dict_keys,index):
    key_in_dict = info_dict.keys()
    file.write(str(index))
    file.write(delimiter)
    for key in dict_keys:
        if key in key_in_dict:
            file.write(str(info_dict[key]).replace("\n"," "))
            file.write(delimiter)
        else:
            file.write("None")
            file.write(delimiter)
    file.write("\n")

In [4]:
proxy_list=list(get_proxies())
base_url = "https://export.arxiv.org/"

In [5]:
# get archives for different fields

def get_arxives(base_url, proxy_list):
    list_href = get_href(base_url, proxy_list)
    outlist = []
    for href in list_href:
        if (href != None and "archive" in href):
            outlist.append(href)
    return outlist

# gets year wise archive for the archives of different fields
def get_years_archives(list_arxives, proxy_list):
    years_archives = []
    for href in list_arxives:
        splitted = href.split('/')
        field = splitted[-1]
        list_href = get_href(base_url+href, proxy_list)
        for links in list_href:
            if ( links != None and field in links and "year" in links):
                years_archives.append(links)
    return years_archives
            
# gets abstract link for the year wise archive
def get_abs_link(list_year_archives, proxy_list):
    abs_link = []
    for href in list_year_archives:
        
        list_href = get_href(base_url+href, proxy_list)
        
        for links in list_href:
            if links!= None:
                splitted = links.split("/")
                number = splitted[-1]
                if ("list" in links and number.isdigit()  ):
                    abs_link.append(links)
    return list(set(abs_link))

# gets the link containing all the abstracts for a year and field
def get_full_abs_link(list_abs, proxy_list):
    abstract_link = []
    for href in list_abs:
        list_href = get_href(base_url+ href, proxy_list)
        link_required = "Garbage"
        for links in list_href:
            if (links != None and href in links and "?show=" in links):
                link_required = links
                
        if link_required == "Garbage":
            link_required = href
        abstract_link.append(link_required)
            
    return list(set(abstract_link))
## get full link for the paper
def get_link_for_paper(full_abs_link, proxy_list):
    full_paper_list = []
    for href in full_abs_link:
        list_href = get_href(base_url + href, proxy_list)
        
        for links in list_href:
            if(links != None and "abs" in links):
                full_paper_list.append(links)
    return list(set(full_paper_list))
    

In [9]:
arxive_list = get_arxives(base_url, proxy_list)

Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89ab5b6978>, 'Connection to 103.96.235.178 timed out. (connect timeout=10)'))


In [10]:
arxive_list

['/archive/astro-ph',
 '/archive/cond-mat',
 '/archive/gr-qc',
 '/archive/hep-ex',
 '/archive/hep-lat',
 '/archive/hep-ph',
 '/archive/hep-th',
 '/archive/math-ph',
 '/archive/nlin',
 '/archive/nucl-ex',
 '/archive/nucl-th',
 '/archive/physics',
 '/archive/quant-ph',
 '/archive/math',
 '/archive/q-bio',
 '/archive/q-fin',
 '/archive/stat',
 '/archive/eess',
 '/archive/econ']

In [11]:
# I am going to generate information for only economics papers
year_arxive_list_econ = get_years_archives(['/archive/econ'], proxy_list)

In [12]:
year_arxive_list_econ

['/year/econ/20',
 '/year/econ/19',
 '/year/econ/18',
 '/year/econ/17',
 '/year/econ/16',
 '/year/econ/15',
 '/year/econ/14',
 '/year/econ/13',
 '/year/econ/12',
 '/year/econ/11',
 '/year/econ/10',
 '/year/econ/09',
 '/year/econ/08',
 '/year/econ/07']

In [21]:
abs_link = get_abs_link(year_arxive_list_econ, proxy_list)

/list/econ/2001
/list/econ/2002
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //year/econ/19 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89ab086eb8>, 'Connection to 103.96.235.178 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //year/econ/19 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aad95978>, 'Connection to 110.74.209.202 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //year/econ/19 (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //year/econ/19 (Caused by ProxyError('Cannot connect to

In [27]:
abs_link

['/list/econ/1711',
 '/list/econ/1612',
 '/list/econ/1807',
 '/list/econ/1508',
 '/list/econ/0708',
 '/list/econ/1504',
 '/list/econ/1604',
 '/list/econ/1603',
 '/list/econ/1908',
 '/list/econ/1709',
 '/list/econ/1104',
 '/list/econ/1004',
 '/list/econ/0806',
 '/list/econ/1804',
 '/list/econ/1511',
 '/list/econ/1512',
 '/list/econ/0912',
 '/list/econ/1305',
 '/list/econ/2001',
 '/list/econ/1606',
 '/list/econ/1811',
 '/list/econ/1802',
 '/list/econ/1909',
 '/list/econ/0910',
 '/list/econ/1710',
 '/list/econ/1901',
 '/list/econ/1207',
 '/list/econ/1903',
 '/list/econ/1607',
 '/list/econ/1702',
 '/list/econ/1911',
 '/list/econ/1707',
 '/list/econ/1910',
 '/list/econ/1106',
 '/list/econ/1510',
 '/list/econ/1806',
 '/list/econ/1805',
 '/list/econ/1601',
 '/list/econ/1803',
 '/list/econ/1605',
 '/list/econ/1411',
 '/list/econ/1808',
 '/list/econ/1609',
 '/list/econ/1501',
 '/list/econ/1608',
 '/list/econ/1904',
 '/list/econ/1907',
 '/list/econ/1405',
 '/list/econ/1311',
 '/list/econ/1412',


In [55]:
abs_link_full = get_full_abs_link(abs_link, proxy_list)

Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1711 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aa5c7940>, 'Connection to 103.96.235.178 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1711 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aa60a080>, 'Connection to 125.26.7.124 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1711 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aa60a630>, 'Connection to 85.10.219.102 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded w

Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1604 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aa5d2a90>, 'Connection to 118.173.233.37 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1604 (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 503 Too many open connections',)))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1604 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aa5d26d8>, 'Connection to 36.89.182.193 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1604 (Caused by ConnectT

Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1305 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aa06b0f0>, 'Connection to 5.63.165.178 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1305 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aa06b9b0>, 'Connection to 103.57.143.246 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/2001 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aa5df978>, 'Connection to 103.96.235.178 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded 

Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1607 (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1607 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aa40cac8>, 'Connection to 86.110.189.118 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1607 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89a9fe5278>, 'Connection to 80.91.118.129 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1607 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSC

Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1806 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89c0986438>, 'Connection to 111.92.164.249 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1806 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89c0986278>, 'Connection to 186.46.3.238 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1601 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aa5ed3c8>, 'Connection to 103.96.235.178 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded 

Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1311 (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 503 Too many open connections',)))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1412 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89ab2ecbe0>, 'Connection to 103.96.235.178 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1905 (Caused by ProxyError('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response',)))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/0704 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPS

Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1210 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aa1e5438>, 'Connection to 36.89.182.193 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1312 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aa283898>, 'Connection to 103.96.235.178 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1312 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aa28cef0>, 'Connection to 92.255.196.91 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded 

Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1602 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aa5f5c18>, 'Connection to 103.96.235.178 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1602 (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')],)",),))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1912 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89ab2e7b70>, 'Connection to 103.96.235.178 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/19

In [6]:
abs_link_full = ['/list/econ/1711',
 '/list/econ/1807?show=58',
 '/list/econ/1612',
 '/list/econ/2002?show=27',
 '/list/econ/1508',
 '/list/econ/0708',
 '/list/econ/1805?show=33',
 '/list/econ/1905?show=86',
 '/list/econ/1504',
 '/list/econ/1604',
 '/list/econ/1911?show=84',
 '/list/econ/1603',
 '/list/econ/1811?show=60',
 '/list/econ/1709',
 '/list/econ/1104',
 '/list/econ/1004',
 '/list/econ/0806',
 '/list/econ/1511',
 '/list/econ/1512',
 '/list/econ/1912?show=85',
 '/list/econ/0912',
 '/list/econ/1907?show=86',
 '/list/econ/1305',
 '/list/econ/1804?show=29',
 '/list/econ/1606',
 '/list/econ/1802',
 '/list/econ/1806?show=45',
 '/list/econ/1809?show=51',
 '/list/econ/0910',
 '/list/econ/1710',
 '/list/econ/1207',
 '/list/econ/1607',
 '/list/econ/1702',
 '/list/econ/1707',
 '/list/econ/1906?show=81',
 '/list/econ/1910?show=93',
 '/list/econ/1106',
 '/list/econ/1510',
 '/list/econ/1601',
 '/list/econ/1803?show=44',
 '/list/econ/1605',
 '/list/econ/1812?show=66',
 '/list/econ/1903?show=69',
 '/list/econ/1904?show=75',
 '/list/econ/1411',
 '/list/econ/1609',
 '/list/econ/1501',
 '/list/econ/1608',
 '/list/econ/1405',
 '/list/econ/1311',
 '/list/econ/1412',
 '/list/econ/0704',
 '/list/econ/1712',
 '/list/econ/1808?show=54',
 '/list/econ/1212',
 '/list/econ/1502',
 '/list/econ/1705',
 '/list/econ/1010',
 '/list/econ/1703',
 '/list/econ/1901?show=55',
 '/list/econ/1206',
 '/list/econ/1210',
 '/list/econ/1801?show=40',
 '/list/econ/1708',
 '/list/econ/1312',
 '/list/econ/1304',
 '/list/econ/1611',
 '/list/econ/1201',
 '/list/econ/1610',
 '/list/econ/1303',
 '/list/econ/1810?show=72',
 '/list/econ/1505',
 '/list/econ/1209',
 '/list/econ/1706',
 '/list/econ/1908?show=98',
 '/list/econ/1012',
 '/list/econ/1309',
 '/list/econ/2001?show=91',
 '/list/econ/1602',
 '/list/econ/1902?show=59',
 '/list/econ/1909?show=99',
 '/list/econ/0904',
 '/list/econ/1704',
 '/list/econ/1105',
 '/list/econ/1507']

In [ ]:
full_paper_link = get_link_for_paper(abs_link_full,proxy_list)

Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1711 (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 503 Too many open connections',)))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1807?show=58 (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 503 Too many open connections',)))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1807?show=58 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7ff1a9e2de80>, 'Connection to 182.53.96.215 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1807?show=58 (Caused by ProxyError('Cannot connect to p

Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1905?show=86 (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 503 Too many open connections',)))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1504 (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 503 Too many open connections',)))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1603 (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 503 Too many open connections',)))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1603 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7ff1a9aa5358>, 'C

Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1912?show=85 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7ff1a9508860>, 'Connection to 187.45.13.190 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1912?show=85 (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1912?show=85 (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 503 Too many open connections',)))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/0912 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(104, 'Connection 

Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1802 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7ff1a8feb860>, 'Connection to 186.250.29.1 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1802 (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 503 Too many open connections',)))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1802 (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1802 (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
Connection Error or Timeout:HTTPSConnec

Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1710 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7ff1a99662e8>, 'Connection to 118.173.232.215 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1710 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7ff1a915c400>, 'Connection to 101.109.143.71 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1710 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7ff1a915c668>, 'Connection to 101.109.143.71 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceed

Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1106 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(104, 'Connection reset by peer')))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1106 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7ff1a9a64518>, 'Connection to 190.12.95.170 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1106 (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //list/econ/1106 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(104, 'Connection reset by peer')))
Connection Error or 

In [ ]:
full_paper_link

In [ ]:
import time
from random import random

output_paper_info = "paper_axrive_info_new.dat"
output_paper_log = "paper_axrive_info_new.log"
output_abstract_path_list = full_paper_link

        
#inFile = open(output_abstract_path,"r")
#lines = inFile.readlines()
#inFile.close()

outfilelog = open(output_paper_log,"w")
outfilelog.close()

outfile = open(output_paper_info,"w")
outfile.write("#")
for key in dict_keys:
    outfile.write(key)
    outfile.write(delimiter)
outfile.write("\n")
outfile.close()
proxy = list(get_proxies())
print("Proxy generated")
len_proxy = len(proxy)

for index, link in enumerate(output_abstract_path_list):
    link = base_url + link
    if ((index+1) % 100 == 0):
        proxy = list(get_proxies())
        print("Proxy generated")


        
        
    try:    
        info_dict = extract_paper_information(link, proxy)
        if(len(list(set(info_dict.values())))==1) and info_dict == None:
            print("%d line not read. Link: %s"%(index,link))
            outfilelog = open(output_paper_log,"a")
            outfilelog.write("%d line not read. Link: %s"%(index,link))
            outfilelog.close()
                        
            continue
        outfile=open(output_paper_info,"a")
        write_to_file(outfile,info_dict,dict_keys,index)
        outfile.close()
        print("%d line read. Link: %s"%(index ,link))


    
    except:
        print("%d link not read. Abnormal Error. Link: %s"%(index,link))
        outfilelog = open(output_paper_log,"a")
        outfilelog.write("%d line not read. Link: %s"%(index,link))
        outfilelog.close()
        continue
        
        
        

Proxy generated
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/1812.11420 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aa320160>, 'Connection to 31.173.94.93 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/1812.11420 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aa3209e8>, 'Connection to 154.126.211.169 timed out. (connect timeout=10)'))
0 line read. Link: https://export.arxiv.org//abs/1812.11420
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/1612.04932 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(104, 'Connection reset by peer')))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max r

Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/1912.02251 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89a9f9f208>, 'Connection to 177.46.147.89 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/1912.02251 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89a9f9f898>, 'Connection to 202.138.242.41 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/1912.02251 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89a9ca30b8>, 'Connection to 125.26.7.85 timed out. (connect timeout=10)'))
7 line read. Link: https://export.arxiv.org//abs/1912.02251
8 line read. Link: https://export.arxiv.org//ab

Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/1803.06108 (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/1803.06108 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aa32e6a0>, 'Connection to 194.114.128.149 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/1803.06108 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aa320390>, 'Connection to 43.252.18.140 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/1803.06108 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPS

24 line read. Link: https://export.arxiv.org//abs/2001.02200
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/2001.03488 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89a9ecfcf8>, 'Connection to 31.173.94.93 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/2001.03488 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89a9cd47f0>, 'Connection to 58.97.72.83 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/2001.03488 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89a9fe8588>, 'Connection to 194.114.128.149 timed out. (connect timeout=10)'))
25 line read. Link: https://export.arxiv.org//

Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/1910.07689 (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
32 line read. Link: https://export.arxiv.org//abs/1910.07689
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/1812.10820 (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/1812.10820 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aaff23c8>, 'Connection to 5.141.244.97 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/1812.10820 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aa27a4a

Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/2002.03569 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aa19f1d0>, 'Connection to 103.129.195.48 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/2002.03569 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89a9f84080>, 'Connection to 177.125.20.35 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/2002.03569 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89a9f843c8>, 'Connection to 193.213.89.72 timed out. (connect timeout=10)'))
40 line read. Link: https://export.arxiv.org//abs/2002.03569
Connection Error or Timeout:HTTPSConnectionP

Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/1810.11397 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aa5d2e80>, 'Connection to 125.27.251.82 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/1810.11397 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aae266a0>, 'Connection to 187.73.68.14 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/1810.11397 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89aae26048>, 'Connection to 77.238.79.111 timed out. (connect timeout=10)'))
45 line read. Link: https://export.arxiv.org//abs/1810.11397
Connection Error or Timeout:HTTPSConnectionPoo

Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/1806.06657 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89ab28ef98>, 'Connection to 195.138.73.54 timed out. (connect timeout=10)'))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/1806.06657 (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/1806.06657 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f89ab28eba8>, 'Connection to 177.53.8.125 timed out. (connect timeout=10)'))
54 line read. Link: https://export.arxiv.org//abs/1806.06657
Connection Error or Timeout:HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: //abs/1807.09595 (Cause